## model load

In [8]:
import librosa
import numpy as np
import math 
import tensorflow as tf

labelset = {"유재석":0, "박명수":1, "정형돈":2, "노홍철":3, "하하":4, 
            "정준하":5, "유인나":6, "아이유":7, "김이나":8, "이수현":9}
labelset_re = {v:k for k,v in labelset.items()}

In [3]:
test_path = "./data/test/mix_ver1.wav"

sample_rate = 16000
y, sr = librosa.load(test_path, sr=sample_rate)
print(y.shape, sr)

time = y.shape[0] / sr
print(time*2)

mfccs_test = []
for i in range(0, math.floor(time/0.5)):
    start = int(i*(sample_rate/2))
    end = int((i+1)*(sample_rate/2))
    features = librosa.feature.mfcc(y=y[start:end], 
                                            sr=sample_rate,
                                            n_mfcc=40, 
                                            hop_length=int(sample_rate*0.01),
                                            n_fft=int(sample_rate*0.02)).T
    mfccs_test.append(features)

(753109,) 16000
94.138625


C:\Users\ASUS\AppData\Roaming\Python\Python39\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


In [4]:
# ndarray 변환
mfccs_test_np = np.array(mfccs_test)
mfccs_test_np.shape

(94, 51, 40)

In [5]:
# reshape
wav_X =  mfccs_test_np.reshape(-1, mfccs_test_np.shape[1], mfccs_test_np.shape[2], 1)
wav_X.shape

(94, 51, 40, 1)

In [7]:
model_cnn = tf.keras.models.load_model('./model_cnn.h5')
pred = model_cnn.predict(wav_X)

3/3 [==============================] - 7s 32ms/step


In [9]:
# 결과값
result = [np.argmax(value) for value in pred]
result[:10]

[9, 8, 9, 8, 8, 8, 8, 8, 3, 3]

In [10]:
# 결과값에 해당하는 이름으로 변환
result_name = [labelset_re[value] for value in result]
result_name[:10]

['이수현', '김이나', '이수현', '김이나', '김이나', '김이나', '김이나', '김이나', '노홍철', '노홍철']

In [11]:
# 각 결과값의 퍼센트
result_per = [np.max(value)*100 for value in pred]
result_per[:10]

[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]

In [12]:
# collections 를 사용하여 파이썬에서 배열에서 발생 횟수 찾기
# collections : 데이터 컬렉션을 저장하는 컨테이너 역할
import collections
collections.Counter(result_per)

Counter({100.0: 85,
         99.99995231628418: 1,
         99.99977350234985: 1,
         99.70706701278687: 1,
         91.13172888755798: 1,
         99.98239874839783: 1,
         99.60312247276306: 1,
         99.99587535858154: 1,
         99.95905756950378: 1,
         99.58181977272034: 1})